# 2. Clean Interim Data

xxxx

In [1]:
import pandas as pd
import datetime as dt
import os

In [2]:
def clean_df(file_path):
    df = pd.read_csv(file_path)
    
    # Get name for the dataframe
    df_name_str = df.iloc[0]['name']
    
    # Calculate the total seconds of sunlight in the day
    df['sunrise'] = pd.to_datetime(df['sunrise'], format='%Y-%m-%dT%H:%M:%S')
    df['sunset'] = pd.to_datetime(df['sunset'], format='%Y-%m-%dT%H:%M:%S')
    df['day_length'] = (df['sunset'] - df['sunrise']).dt.total_seconds()
    
    # Convert 'datetime' column to datetime datatype
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Extract year and month
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    
    # Group by year and calculate average values for each year
    yearly_avg = df.groupby('year').agg({
        'day_length': 'mean',
        'tempmax': 'mean',
        'tempmin': 'mean',
        'temp': 'mean',
        'dew': 'mean',
        'humidity': 'mean',
        'precip': 'mean',
        'precipprob': 'mean',
        'precipcover': 'mean',
        'snowdepth': 'mean',
        'windgust': 'mean',
        'windspeed': 'mean',
        'windspeedmax': 'mean',
        'windspeedmean': 'mean',
        'windspeedmin': 'mean',
        'sealevelpressure': 'mean',
        'cloudcover': 'mean',
        'solarradiation': 'mean',
        'solarenergy': 'mean',
        'uvindex': 'mean'
    }).reset_index()
    
    # Calculate monthly averages for each feature
    monthly_avg = df.groupby(['year', 'month']).mean(numeric_only=True).unstack()
    
    # Calculate yearly averages for each feature
    yearly_avg = df.groupby('year').mean(numeric_only=True)
    
    # Drop redundant columns
    yearly_avg.drop(columns=['day_length', 'month', 'tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'snowdepth', 'windgust', 'windspeed', 'windspeedmax', 'windspeedmean', 'windspeedmin', 'sealevelpressure', 'cloudcover', 'solarradiation', 'solarenergy', 'uvindex'], inplace=True)
    
    # Combine monthly and yearly averages
    final_df = pd.concat([yearly_avg, monthly_avg], axis=1)
    
    # Rename columns for clarity
    final_df.columns = [f'{col[0]}_{col[1]:02d}' if isinstance(col, tuple) else col for col in final_df.columns]

    # Turn the index column into a real column
    final_df.reset_index(inplace=True)

    # Add the name of the dataframe
    final_df.insert(0, 'location', df_name_str)

    return final_df

In [3]:
def process_weather_files(dir_path):
    # Check if the path exists
    if not os.path.exists(dir_path):
        print(f"Path '{dir_path}' does not exist.")
        return

    df_dict = {}
    
    # Iterate through files in the directory
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            # Construct the full file path
            file_path = os.path.join(dir_path, filename)
            
            # Read the CSV file into a pandas DataFrame
            df = clean_df(file_path)

            # Append the cleaned df to the list
            df_dict[filename] = df
    
    # Return the list of cleaned dfs
    return df_dict

In [4]:
dir_paths = ["../data/interim/representative_locations/weather/src",
             "../data/interim/main_locations/weather/src"]

OUTPUT_PATHS = ["../data/interim/representative_locations/weather/monthly_agg/",
                "../data/interim/main_locations/weather/monthly_agg/"]

In [5]:
for index in range(0, len(dir_paths)):
    # Process the raw data
    df_dict = process_weather_files(dir_paths[index])
    
    # Output the aggregated files
    for file_name in df_dict:
        df_dict[file_name].to_csv(f"{OUTPUT_PATHS[index]}{file_name}", index=False)